In [67]:
import networkx as nx
from graph_utils import plot_graph
import numpy as np
import pandas as pd
from gurobipy import GRB, Model

In [68]:
static = pd.read_csv('static_with_t.csv')
detected = pd.read_csv('detection_graph.csv')

In [69]:
level_order = [
    "root_cross_right",
    "root_cross_left",
    "root_cross_right2",
    "conn_k",
    "cross_e",
    "cross_bag_br",
    "cross_ij1",
    "cross_e_c",
    "cross_f",
    "bag_b_r",
    "bag_a_r",
    "conn_b",
    "cross_ij2",
    "conn_e",
    "conn_c",
    "cross_k",
    "cross_f30",
    "conn_i",
    "conn_j",
    "box",
    "conn_a",
    "conn_f_30",
    "cross_f37",
    "conn_f_60",
    "bag_b_l",
    "cross_gh",
    "conn_f_37",
    "conn_d",
    "conn_g",
    "conn_h",
    "bag_a_l",
]

In [70]:
quadratic_model = Model('quadratic')

In [71]:
variables = {}
for name in level_order:
    variables[f"{name}_x"] = quadratic_model.addVar(
        vtype=GRB.CONTINUOUS, lb=-250, ub=250, name=f"{name}_x"
    )
    variables[f"{name}_y"] = quadratic_model.addVar(
        vtype=GRB.CONTINUOUS, lb=-250, ub=250, name=f"{name}_y"
    )
    variables[f"{name}_z"] = quadratic_model.addVar(
        vtype=GRB.CONTINUOUS, lb=-250, ub=250, name=f"{name}_z"
    )
    
    
        

In [72]:
obj_fn = 0
for name in level_order:
    if name == "root":
        continue
    node = detected[detected["name"] == name]
    if node.shape[0] != 0:
        detected_x = node["x"].values[0]
        detected_y = node["y"].values[0]
        detected_z = node["z"].values[0]
        parent = static[static["name"] == name]["parent"].values[0]
        term = (
            (variables[f"{name}_x"] - detected_x) ** 2
            + (variables[f"{name}_y"] - detected_y) ** 2
            + (variables[f"{name}_z"] - detected_z) ** 2
        )
        obj_fn += term

In [73]:
for name in level_order:
    node_info = static[static["name"] == name]
    parent = node_info["parent"].values[0]
    distance_to_parent = node_info["distance_to_parent"].values[0]
    if parent == "root":
        quadratic_model.addQConstr(
            (
                variables[f"{name}_x"] ** 2
                + variables[f"{name}_y"] ** 2
                + variables[f"{name}_z"] ** 2
            )
            <= distance_to_parent**2
        )
    else:
        quadratic_model.addQConstr(
            (
                (variables[f"{name}_x"] - variables[f"{parent}_x"]) ** 2
                + (variables[f"{name}_y"] - variables[f"{parent}_y"]) ** 2
                + (variables[f"{name}_z"] - variables[f"{parent}_z"]) ** 2
            )
            <= distance_to_parent**2
        )

In [74]:
quadratic_model.setObjective(obj_fn, GRB.MINIMIZE)


In [75]:
quadratic_model.optimize()


Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: 11th Gen Intel(R) Core(TM) i5-11500H @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 0 rows, 93 columns and 0 nonzeros
Model fingerprint: 0xa2fa8c1e
Model has 54 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [6e+00, 4e+02]
  QObjective range [2e+00, 2e+00]
  Bounds range     [2e+02, 2e+02]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 93 columns
Presolve time: 0.01s
Presolve: All rows and columns removed

Barrier solved model in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective 0.00000000e+00


In [76]:
#print all variable values
for v in quadratic_model.getVars():
    print(v.varName, v.x)

root_cross_right_x -250.0
root_cross_right_y -250.0
root_cross_right_z -250.0
root_cross_left_x -250.0
root_cross_left_y -250.0
root_cross_left_z -250.0
root_cross_right2_x -250.0
root_cross_right2_y -250.0
root_cross_right2_z -250.0
conn_k_x -0.0
conn_k_y -104.0
conn_k_z -10.0
cross_e_x -250.0
cross_e_y -250.0
cross_e_z -250.0
cross_bag_br_x -250.0
cross_bag_br_y -250.0
cross_bag_br_z -250.0
cross_ij1_x -250.0
cross_ij1_y -250.0
cross_ij1_z -250.0
cross_e_c_x -250.0
cross_e_c_y -250.0
cross_e_c_z -250.0
cross_f_x -250.0
cross_f_y -250.0
cross_f_z -250.0
bag_b_r_x 16.0
bag_b_r_y -105.0
bag_b_r_z -5.0
bag_a_r_x 13.0
bag_a_r_y -150.0
bag_a_r_z -0.0
conn_b_x 21.0
conn_b_y -86.0
conn_b_z -0.0
cross_ij2_x -250.0
cross_ij2_y -250.0
cross_ij2_z -250.0
conn_e_x -5.0
conn_e_y -87.0
conn_e_z -12.0
conn_c_x -3.0
conn_c_y -87.0
conn_c_z -12.0
cross_k_x -250.0
cross_k_y -250.0
cross_k_z -250.0
cross_f30_x -250.0
cross_f30_y -250.0
cross_f30_z -250.0
conn_i_x 16.0
conn_i_y -75.0
conn_i_z 3.0
conn_j_

In [77]:
import networkx as nx
from graph_utils import plot_graph
import numpy as np
import pandas as pd
from transformation_utils import (
    get_rotation,
    rotate_graph,
    get_rotation_nn,
    optim_rotation,
)

G1 = nx.Graph()
for i in range(len(static)):
    G1.add_node(static['name'][i], pos=(static['x'][i], static['z'][i], static['y'][i]))
for i in range(len(static)):
    if static['parent'][i] != 0 and static['parent'][i] != '0':
        G1.add_edge(static['name'][i], static['parent'][i])


G2 = nx.Graph()
for name in level_order:
    # if x and y and z are  different than 250 or -250
    if variables[f"{name}_x"].x != 250 and variables[f"{name}_x"].x != -250:
        # print(name, variables[f"{name}_x"].x, variables[f"{name}_y"].x, variables[f"{name}_z"].x)
        G2.add_node(
            name,
            pos=(
                variables[f"{name}_x"].x,
                variables[f"{name}_z"].x,
                variables[f"{name}_y"].x,
            ),
        )




plot_graph(static_graph=G1, detected_graph=G2, name="qcqp2", save=True)

Graph saved to qcqp2.html


In [78]:
#print cross_bag_br_x
print(variables["cross_bag_br_x"].x)

-250.0
